In [63]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:

from pprint import pp
from libs.corpus_nlp.bnc_corpus.corpus_bnc import xml_file_to_dict, BNCFileWrapper, bnc_corpus_sent_join


In [65]:
# example usage of BNCCorpusReader

# bnc_reader = BNCCorpusReader(
#     root='../data/corpuses/BNC/Texts/', fileids=r'[A-K]/\w*/\w*\.xml')


In [70]:
# print the xml structure for exploration

xml_file = '../data/corpuses/BNC/Texts/A/A4/A4W.xml'
# result_dict = xml_file_to_dict(xml_file)
# # if you only want the header and not the sentences
result_dict = xml_file_to_dict(xml_file, stop_key='wtext')
with open("./out.txt", 'w') as f:
    print(f)
    pp(result_dict, f)


Stopping parse at wtext
<_io.TextIOWrapper name='./out.txt' mode='w' encoding='UTF-8'>


In [155]:
# count files by description type (genre)

from collections import Counter
from pathlib import Path

c = Counter()
total_ct = 0
file_ct = 0

xml_root = Path('../data/corpuses/BNC/Texts/A/')
for f in xml_root.glob("**/*.xml"):
    file_ct += 1
    wrap = BNCFileWrapper(".", str(f))
    genre = wrap.info.profile_text_class_code['_text']
    c[genre] += 1
    total_ct += wrap.info.sent_ct

pp(c)
pp(total_ct)
pp(file_ct)



Counter({'W newsp brdsht nat: misc': 95,
         'W misc': 62,
         'W fict prose': 57,
         'W newsp brdsht nat: arts': 51,
         'W newsp brdsht nat: report': 49,
         'W newsp brdsht nat: commerce': 44,
         'W pop lore': 37,
         'W newsp brdsht nat: social': 36,
         'W newsp brdsht nat: science': 29,
         'W nonAc: humanities arts': 28,
         'W biography': 25,
         'W newsp brdsht nat: sports': 24,
         'W nonAc: soc science': 20,
         'W ac:humanities arts': 18,
         'W nonAc: polit law edu': 16,
         'W ac:polit law edu': 13,
         'W newsp brdsht nat: editorial': 12,
         'W nonAc: nat science': 11,
         'W commerce': 9,
         'W ac:soc science': 9,
         'W religion': 7,
         'W nonAc: medicine': 7,
         'W advert': 4,
         'W ac:nat science': 3,
         'W instructional': 3,
         'W institut doc': 3,
         'W letters prof': 1,
         'W nonAc: tech engin': 1})
753221
674


In [150]:
# example info of the BNC file

bnc_wrap = BNCFileWrapper('../data/corpuses/BNC/Texts/', 'A/A4/A4W.xml')
pp(bnc_wrap.info)


BNCFileInfo(file_id_full='A/A4/A4W.xml',
            file_id='A4W',
            file_path='/Users/jsrozner/docs_local/_programming/research_constructions/constructions_repo/data/corpuses/BNC/Texts/A/A4/A4W.xml',
            edition='BNC XML Edition, December 2006',
            extent='3792 tokens; 3827 w-units; 178 s-units',
            source_desc={'title': {'_text': 'Independent, electronic edition '
                                            'of 1989-10-12: Focus section.'},
                         'imprint': {'n': 'NEWSPA2',
                                     'publisher': {'_text': 'Newspaper '
                                                            'Publishing plc'},
                                     'pubPlace': {'_text': 'London'},
                                     'date': {'_text': '1989',
                                              'value': '1989'}}},
            title_stmt='Independent, electronic edition of 1989-10-12: Focus '
                       'section. 

In [154]:
# sample run using the MLM scorer

from lib.mlm import MLMScorer
mlm_scorer = MLMScorer('roberta-large')

for s in bnc_wrap.sentences()[:2]:
    # print(s)
    sent = bnc_corpus_sent_join(s)
    # print(sent)
    print(mlm_scorer.print_preds_all(sent, 10))


Stuck
----------------------------------------
fast
Entropy: 1.23; HHI: 0.651; step_score: 0.9698784351348877 
 deep - 61.76 - 1.0
 down - 58.76 - 0.05
 firmly - 58.13 - 0.027
 deeper - 57.66 - 0.017
 forever - 57.54 - 0.015
 back - 57.5 - 0.014
 permanently - 57.0 - 0.009
 low - 56.95 - 0.008
 again - 56.79 - 0.007
 in - 56.75 - 0.007
----------------------------------------
in
Entropy: 0.13; HHI: 0.959; step_score: 0.00017656071577221155 
 in - 69.43 - 1.0
 to - 65.11 - 0.013
 into - 63.89 - 0.004
 on - 62.46 - 0.001
 under - 62.01 - 0.001
 within - 61.7 - 0.0
 by - 61.01 - 0.0
 beneath - 60.94 - 0.0
 against - 60.72 - 0.0
 with - 60.66 - 0.0
----------------------------------------
the
Entropy: 0.09; HHI: 0.98; step_score: 0.0 
 the - 68.77 - 1.0
 a - 63.48 - 0.005
 its - 61.08 - 0.0
 financial - 60.25 - 0.0
 deep - 60.12 - 0.0
 this - 59.5 - 0.0
 bureaucratic - 59.04 - 0.0
 muddy - 58.96 - 0.0
 automotive - 58.9 - 0.0
 mud - 58.87 - 0.0
----------------------------------------
mire

In [11]:
# check that tokenization is working as expected

from lib.exp_common.mlm_align_tokens import reassembled_words_from_tokens_roberta
sent = bnc_corpus_sent_join(bnc_wrap.sentences()[3])
print(sent)
input_ids: List[List[int]] = mlm_scorer.tokenizer.encode(sent, return_tensors='pt')  
tokens: List[str] = mlm_scorer.tokenizer.convert_ids_to_tokens(input_ids[0])  # get 1st sent in batch
print(tokens)
tokenized_words = reassembled_words_from_tokens_roberta(tokens)
print(tokenized_words)


 FORD'S ageing and ramshackle Dagenham site in Essex, once the jewel in the company's overseas operations, enters the 1990s with its very survival uncertain.
['<s>', 'ĠFOR', 'D', "'", 'S', 'Ġageing', 'Ġand', 'Ġram', 'sh', 'ackle', 'ĠD', 'agen', 'ham', 'Ġsite', 'Ġin', 'ĠEssex', ',', 'Ġonce', 'Ġthe', 'Ġjewel', 'Ġin', 'Ġthe', 'Ġcompany', "'s", 'Ġoverseas', 'Ġoperations', ',', 'Ġenters', 'Ġthe', 'Ġ1990', 's', 'Ġwith', 'Ġits', 'Ġvery', 'Ġsurvival', 'Ġuncertain', '.', '</s>']
[TokenizedWord(str_rep='<s>', tokens=['<s>'], tok_idx_start=0), TokenizedWord(str_rep='', tokens=[], tok_idx_start=1), TokenizedWord(str_rep='ĠFORDS', tokens=['ĠFOR', 'D', 'S'], tok_idx_start=1), TokenizedWord(str_rep='Ġageing', tokens=['Ġageing'], tok_idx_start=5), TokenizedWord(str_rep='Ġand', tokens=['Ġand'], tok_idx_start=6), TokenizedWord(str_rep='Ġramshackle', tokens=['Ġram', 'sh', 'ackle'], tok_idx_start=7), TokenizedWord(str_rep='ĠDagenham', tokens=['ĠD', 'agen', 'ham'], tok_idx_start=10), TokenizedWord(str_rep=

{'A46': {'file_id_full': '../data/corpuses/BNC/Texts/A/A4/A46.xml',
         'file_id': 'A46',
         'file_path': '/Users/jsrozner/docs_local/_programming/research_constructions/constructions_repo/data/corpuses/BNC/Texts/A/A4/A46.xml',
         'edition': 'BNC XML Edition, December 2006',
         'extent': '12394 tokens; 12506 w-units; 579 s-units',
         'source_desc': {'title': {'_text': 'Independent, electronic edition '
                                            'of 1989-10-10: Foreign news '
                                            'pages.'},
                         'imprint': {'n': 'NEWSPA2',
                                     'publisher': {'_text': 'Newspaper '
                                                            'Publishing plc'},
                                     'pubPlace': {'_text': 'London'},
                                     'date': {'_text': '1989',
                                              'value': '1989'}}},
         'title_stmt': 'Indepen